In [5]:
# Dans cette partie, on utilise le module csv pour lire les données d'une base de données de test.
# Cette base est utilisée pour entraîner/tester un réseau de neurone à apprentissage supervisé.
fichier = open("iris.csv")
import csv
lecteur = csv.reader(fichier)
for ligne in lecteur:
    if(lecteur.line_num == 1):
        n = int(ligne[0])
        p = int(ligne[1])
        c = int(ligne[2])
        x = []
        d = []
    else:
        # print(ligne): chaque ligne est une liste de chaînes de caractères
        x.append([float(ligne[j]) for j in range(p)])
        tmp = [0 for i in range(c)]
        tmp[int(ligne[p])-1] = 1
        d.append(tmp)

print(x[50])
print(d[50])



[7.0, 3.2, 4.7, 1.4]
[0, 1, 0]


In [6]:
# Implémentation d'un neurone formel
from random import random
from math import exp
class Neurone:
    def __init__(self, nbrPoids):
        self.nbrPoids = nbrPoids
        self.sortie = random()
        self.poidsSynap = [random() for i in range(nbrPoids)]
        self.biais = random()
        self.entree = []
        self.delta = 0.
    def chargeEnt(self, vectEnt):
        self.entree = [vectEnt[i] for i in range(self.nbrPoids)]
    def calculSortie(self): #Nécissite l'exécution de la fonction chargeEnt
        self.sortie = self.biais
        for i in range(self.nbrPoids):
            self.sortie += self.poidsSynap[i]*self.entree[i]
        self.sortie = 1./(1.+exp(-self.sortie))
    def ajustPoids(self, eta, delta):
        self.delta = delta
        for i in range(self.nbrPoids):
            self.poidsSynap[i] += eta*self.delta*self.entree[i]
        self.biais += eta*delta


nrn = Neurone(4)
print(x[20])
nrn.chargeEnt(x[20])
nrn.calculSortie()
print(nrn.sortie)
for i in range(10000):
    nrn.calculSortie()
    delta = (0. - nrn.sortie)*nrn.sortie*(1. - nrn.sortie)
    nrn.ajustPoids(0.2, delta)
    print(nrn.sortie)



[5.4, 3.4, 1.7, 0.2]
0.9973778365391702
0.9973778365391702
0.997316206680168
0.9972516402607778
0.9971839237860589
0.9971128226677103
0.9970380785652339
0.9969594063161508
0.996876490379482
0.9967889807003474
0.9966964878831192
0.9965985775349188
0.9964947636088707
0.9963845005353675
0.9962671738769872
0.996142089174949
0.996008458567136
0.995865384642918
0.9957118408488168
0.9955466475582111
0.9953684426490736
0.9951756450694038
0.9949664093718497
0.9947385685103403
0.99448956122823
0.9942163390028095
0.9939152455513455
0.9935818590478607
0.9932107829708078
0.9927953651298638
0.9923273146359864
0.9917961712406921
0.9911885568705646
0.9904870987216402
0.9896688448391957
0.9887028736694368
0.9875465831732353
0.9861397390185713
0.9843945622035252
0.982178479650519
0.9792825128164633
0.9753596573460354
0.9697954623181667
0.9614102437448482
0.947692544392514
0.9225403393667734
0.8686036661161533
0.731725374891974
0.43062236350628247
0.22755521263937575
0.17088726975617644
0.142391626863723

In [7]:
# Implémentation d'une couche neuronale
class Couche:
    def __init__(self, nbrNrn, nbrPoids):
        self.nbrNrn = nbrNrn
        self.nbrPoids = nbrPoids
        self.neurone = [Neurone(nbrPoids) for i in range(nbrNrn)]
    def chargeEnt(self, vectEnt):
        for i in range(self.nbrNrn):
            self.neurone[i].chargeEnt(vectEnt)
    def calculSortie(self): # Prérequis: exécution de la fonction chargerEnt
        for i in range(self.nbrNrn):
            self.neurone[i].calculSortie()
    def affichSortie(self):
        print([self.neurone[i].sortie for i in range(self.nbrNrn)])
    def ajustPoids(self, eta, delta):
        for i in range(self.nbrNrn):
            self.neurone[i].ajustPoids(eta, delta[i])

cch = Couche(3,p)
for i in range(3):
    print(str(cch.neurone[i].poidsSynap) + " " + str(cch.neurone[i].biais))
print()
print(x[60])
print()
cch.chargeEnt(x[0])
cch.calculSortie()
cch.affichSortie()

for i in range(1000):
    cch.calculSortie()
    for j in range(cch.nbrNrn):
        delta = (d[0][j] - cch.neurone[j].sortie)*cch.neurone[j].sortie*(1. - cch.neurone[j].sortie)
        cch.neurone[j].ajustPoids(0.2, delta)

cch.calculSortie()
cch.affichSortie()



[0.35383943293824316, 0.6105435687594412, 0.8137383074774122, 0.24489350908565144] 0.2706127147307986
[0.4480536533646148, 0.3601906713995695, 0.949128473287032, 0.14850029851052682] 0.12734681540282844
[0.4923170047191371, 0.5718284656682565, 0.4500341130333293, 0.3707194779139973] 0.1535517610258086

[5.0, 2.0, 3.5, 1.0]

[0.9955051081432661, 0.9935137781584786, 0.9953672437677795]
[0.9961034842455251, 0.007984956823329644, 0.00801723502241268]


In [8]:
class PMC:
    def __init__(self, nbrCch, nbrNrn, nbrPoids): #nbrNrn: tab, nbrPoids: tab
        #la première composante du vect nbrNrn contient le nombre de neurones de la première couche
        self.couche = [Couche(nbrNrn[i], nbrPoids[i]) for i in range(nbrCch)]
        self.nbrCch = nbrCch
        self.nbrNrn = [nbrNrn[i] for i in range(self.nbrCch)]
        self.nbrPoids = [nbrPoids[i] for i in range(self.nbrCch)]
    def propager(self, vectEnt): # c'est calculerSortie pour le réseau de neurones
        self.couche[0].chargeEnt(vectEnt)
        self.couche[0].calculSortie()
        for i in range(1, self.nbrCch):
            sortCchPrec = [self.couche[i-1].neurone[j].sortie for j in range(self.nbrNrn[i-1])]
            self.couche[i].chargeEnt(sortCchPrec)
            self.couche[i].calculSortie()
    def affichSortie(self):
        self.couche[self.nbrCch-1].affichSortie()
    def entrainer(self, X, D, eta, tmax):
        for r in range(tmax):
            for q in range(len(X)):
                # Propager les signaux d'entrée pour chaque exemple
                self.propager(X[q])
                # Ajuster les poids synaptiques des neurones de la couche de sortie
                delta = [(D[q][j] - self.couche[self.nbrCch-1].neurone[j].sortie)*self.couche[self.nbrCch-1].neurone[j].sortie*(1. - self.couche[self.nbrCch-1].neurone[j].sortie) for j in range(self.couche[self.nbrCch-1].nbrNrn)]
                for j in range(self.couche[self.nbrCch-1].nbrNrn):
                    self.couche[self.nbrCch-1].neurone[j].ajustPoids(eta, delta[j])
                # Ajuster les poids synaptiques des neurones des couches cachées et de la couche d'entrée
                for s in range(self.nbrCch-2, -1, -1):
                    delta = [random() for i  in range(self.couche[s].nbrNrn)]
                    for k in range(self.couche[s].nbrNrn):
                        delta[k] = 0. 
                        for i in range(self.couche[s+1].nbrNrn):
                            delta[k] += self.couche[s+1].neurone[i].poidsSynap[k]*self.couche[s+1].neurone[i].delta
                        delta[k] *= self.couche[s].neurone[k].sortie*(1. - self.couche[s].neurone[k].sortie)
                    for j in range(self.couche[s].nbrNrn):
                        self.couche[s].neurone[j].ajustPoids(eta, delta[j])



pmc = PMC(2, [3, c], [p, 3])
pmc.entrainer(x, d, 0.2, 10000)
for q in range(len(x)):
    pmc.propager(x[q])
    pmc.couche[pmc.nbrCch-1].affichSortie()


[0.9965740403404254, 0.0020669332460393564, 0.0032041709644029847]
[0.9963744813642376, 0.002175600903499182, 0.0032315997428262644]
[0.9965190602609972, 0.0020972075348272778, 0.0032134530141840504]
[0.9962587002633271, 0.0022383605035409806, 0.003246768119164249]
[0.9965850421586339, 0.002060956001247606, 0.00320279761927488]
[0.9965369574701521, 0.0020868755218851074, 0.0032076741250148624]
[0.9964959585305594, 0.0021096409590425852, 0.0032157478103613663]
[0.996503902932966, 0.002105134168240877, 0.003213634416361217]
[0.9961609724991299, 0.0022914578137772455, 0.003260676334521706]
[0.9963872539514815, 0.002168652556189429, 0.003229826014621437]
[0.9965907648296647, 0.002057666846755256, 0.0032010379354138856]
[0.9964049278944964, 0.00215893348872284, 0.003226800056636432]
[0.9964104516619512, 0.002156230770681661, 0.0032277316679569473]
[0.9965426278678707, 0.0020851934818443994, 0.0032150135598405655]
[0.9966346787489584, 0.0020336611822977376, 0.003194767136104488]
[0.996629909